In [ ]:
# TeamNM3 Movie Recommendation System

© Explore Data Science Academy

---
### Honour Code

We, **Murtala Umar**,
   **Harmony Odumuku**,
   **Njoku Okechukwu**,
   **Akinbo Akin Taylor**,
   **Prince Charles**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Overview:

The rapid growth of data collection has led to a new era of information. Data is being used to create more efficient systems and this is where Recommendation Systems come into play. Recommendation Systems are a type of information filtering systems as they improve the quality of recommendations and search results providing items that are more relevant to the user.

In today’s technology driven world, recommender systems are socially and economically critical to ensure that individuals can make optimised choices surrounding the content they engage with on a daily basis. One application where this is especially true is movie recommendations; where intelligent algorithms can help viewers find great titles from tens of thousands of options.


<a id="cont"></a>

## Table of Contents

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#four>4. Data Engineering</a>

<a href=#five>5. Modeling</a>

<a href=#six>6. Model Performance</a>

<a href=#seven>7. Model Explanations</a>

 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Importing Packages ⚡ |
| :--------------------------- |
| In this section you are required to import, and briefly discuss, the libraries that will be used throughout your analysis and modelling. |

---

# Libraries for data loading, data manipulation and data visulisation
import pandas as pd

# Libraries for data preparation and model building
# Data Manipulation
import numpy as np
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline

# Setting global constants to ensure notebook results are reproducible
# PARAMETER_CONSTANT = ###

<a id="two"></a>
## 2. Loading the Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---

| ⚡ Description: Loading the data ⚡                                                                                                                                          |
|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Here we loaded the movie data available to us for the purposes of the project.|

---

### Use this if you have files locally in datasets subdirectory

movies = pd.read_csv("datasets/movies.csv")
imdb_data = pd.read_csv("datasets/imdb_data.csv")# load the data
genome_scores = pd.read_csv('datasets/genome_scores.csv')
links = pd.read_csv('datasets/links.csv')
tags = pd.read_csv('datasets/tags.csv')
genome_tags = pd.read_csv('datasets/genome_tags.csv')

train = pd.read_csv("datasets/train.csv")
test = pd.read_csv("datasets/test.csv")

### Use this if you do not have files locally (in datasets subdirectory)
"""
base_url = "https://storage.googleapis.com/kagglesdsdata/competitions/36029/3495179/genome_tags.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1657753808&Signature=ZksGirewrm6%2FMk3KGSotOFO3KFCpTSYEduedm4W93dFFU%2BYWC7l2Q9hbAslYyMU97uGnLfRjONQwLOuWPZps1MiXUcMQfYdPg3sdGLO3%2BeMDWZLhOx2HToMMWGXkyyVuZGDDa78MKwNXmyNqxiQvRJxni0MIlkRskAVeLpQhrkP%2B7zK77bzE54OtnAi1VfIIl1WXKsMG76aVuDBvVGh%2BlyJspAMK%2BVtX3ojgPzro9kVGrxgOIxxodyCvG5xfGFVzqAJN8HsX8eBTNFM8fqpbfhcYlOpT3xbZgkpSmMnMOWZbRQIaVdKjUBjccmaNKQVMADNJViEbhyuCCGq1rEBHaw%3D%3D&response-content-disposition=attachment%3B+filename%3D"
movies1 = pd.read_csv(base_url+"/movies.csv")
imdb_data = pd.read_csv(base_url+"/imdb_data.csv")# load the data
genome_scores = pd.read_csv(base_url+"/genome_scores.csv")
links = pd.read_csv(base_url+"/links.csv")
tags = pd.read_csv(base_url+"/tags.csv")
genome_tags = pd.read_csv(base_url+"/genome_tags.csv")

train = pd.read_csv(base_url+"/train.csv")
test = pd.read_csv(base_url+"/test.csv")
"""

movies.head()

movies.shape

movies.info()

imdb_data.head()

imdb_data.info();

imdb_data.shape


genome_scores.head()

genome_scores.shape


links.head()

links.shape


tags.head()

tags.shape


genome_tags.head()

genome_tags.shape

# existing files: 

"""
1. movies.csv
2. imdb_data.csv
3. genome_scores.csv
4. links.csv
5. tags.csv
6. genome_tags.csv
"""

# dataset = pd.merge(movies,imdb_data, on="movieId")
# merged_df = pd.concat([movies, imdb_data, genome_scores, links, tags, genome_tags])
# merged_df

# data = movies.merge(imdb_data, on='movieId')
# data = data.merge(links, on='movieId')
# data = data.merge(genome_scores, on="movieId")
# 
# data

data = tags.merge(movies, on='movieId')
data = data.merge(links, on='movieId')


data.head()


scores.head()



scores.shape

# check for datasets info
datasets.info

scores = pd.read_csv('datasets/genome_scores.csv')
scores.head()


train


test

# check for datasets info
datasets.info

test = pd.read_csv("datasets/test.csv")
test

data.columns

train = pd.read_csv("datasets/train.csv")
train

test = pd.read_csv("datasets/test.csv")
test

<a id="three"></a>
## 3. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Exploratory data analysis ⚡                                                                                                                                           |
|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| To understand our data, we are taken an indept exploration of our data here.|

---


# look at data statistics

# plot relevant feature interactions

# evaluate correlation

# have a look at feature distributions

<a id="four"></a>
## 4. Data Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Data engineering ⚡ |
| :--------------------------- |
| In this section you are required to: clean the dataset, and possibly create new features - as identified in the EDA phase. |

---



# remove missing values/ features

# create new features

# engineer existing features

<a id="five"></a>
## 5. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Modelling ⚡ |
| :--------------------------- |
| In this section, you are required to create one or more regression models that are able to accurately predict the thee hour load shortfall. |

---

# split data

# create targets and features dataset

# create one or more ML models

# evaluate one or more ML models

<a id="six"></a>
## 6. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model performance ⚡ |
| :--------------------------- |
| In this section you are required to compare the relative performance of the various trained ML models on a holdout dataset and comment on what model is the best and why. |

---

# Compare model performance

# Choose best model and motivate why it is the best choice

<a id="seven"></a>
## 7. Model Explanations
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model explanation ⚡ |
| :--------------------------- |
| In this section, you are required to discuss how the best performing model works in a simple way so that both technical and non-technical stakeholders can grasp the intuition behind the model's inner workings. |

---

# discuss chosen methods logic